# Using Yahoo Finance to calc RSI and MFI

## Imports and Constants

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
import yfinance as yf
import mplfinance as mpf

In [28]:
candle_period = '1wk'
rolling_period = 14
markets = ['^SPX', '^NDX', '^FTLC', 'GBRE.L', 'TECW.L', 'ESIC.DE', 'SJPA.L',  'VDPG.L', 'XDW0.L', 'UC15.L']

## Get OHLCV

In [29]:
data = yf.download('^SPX', interval = candle_period)
data

[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
1927-12-26,17.660000,17.660000,17.660000,17.660000,17.660000,0
1928-01-02,17.760000,17.760000,17.549999,17.660000,17.660000,0
1928-01-09,17.500000,17.580000,17.350000,17.580000,17.580000,0
1928-01-16,17.290001,17.480000,17.260000,17.480000,17.480000,0
1928-01-23,17.639999,17.709999,17.520000,17.690001,17.690001,0
...,...,...,...,...,...,...
2023-12-11,4593.390137,4738.569824,4593.390137,4719.189941,4719.189941,27228260000
2023-12-18,4725.580078,4778.009766,4697.819824,4754.629883,4754.629883,18766580000
2023-12-25,4758.859863,4793.299805,4751.990234,4769.830078,4769.830078,11087280000


In [6]:
data = yf.download('TECW.L', interval = candle_period)
data

[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2022-04-04,86.909927,88.050003,84.372543,84.014999,84.014999,176
2022-04-11,83.344223,83.344223,81.109428,81.650002,81.650002,396
2022-04-18,81.650002,82.473549,80.459999,80.934998,80.934998,281
2022-04-25,79.489098,83.239998,79.147079,82.050003,82.050003,809
2022-05-02,82.050003,83.911911,79.347954,81.184998,81.184998,823
...,...,...,...,...,...,...
2023-12-11,103.555069,106.480003,103.537216,105.919998,105.919998,1426
2023-12-18,105.597397,107.099998,104.800003,105.349998,105.349998,1573
2023-12-25,105.349998,106.660004,105.151154,106.000000,106.000000,707


In [ ]:
type(data)

In [ ]:
data.describe()

In [ ]:
data.tail()

## RSI

In [ ]:
data['Change'] = data['Adj Close'].diff()
data['Gain'] = data['Change'].mask(data['Change'] < 0, 0.0)
data['Loss'] = -data['Change'].mask(data['Change'] > 0, -0.0)

In [ ]:
data['Avg Gain'] = data['Gain'].rolling(rolling_period).mean()
data['Avg Loss'] = data['Loss'].rolling(rolling_period).mean()

In [ ]:
data['Avg Gain'] = data['Gain'].ewm(com=rolling_period, adjust=False).mean()
data['Avg Loss'] = data['Loss'].ewm(com=rolling_period, adjust=False).mean()

In [ ]:
data['RS'] = data['Avg Gain'] / data['Avg Loss']
data['RSI'] = 100 - (100 / (1 + data['RS']))
data

In [ ]:
mpf.plot(data, type='candle', style='yahoo', volume=True)

In [ ]:
data[['RSI']].iloc[-52:].plot()

## MFI

In [ ]:
data['Typical Price'] = (data['Adj Close'] + data['High'] + data['Low']) / 3
data['Money Flow'] = data['Typical Price'] * data['Volume']

In [ ]:
def mf_dir(typical_price,money_flow):

    positive_flow = []
    negative_flow = []
    
    for i in range(1, len(typical_price)):
        if typical_price[i] > typical_price[i-1]:
            positive_flow.append(money_flow[i-1])
            negative_flow.append(0)
            
        elif typical_price[i] < typical_price[i-1]:
            negative_flow.append(money_flow[i-1])
            positive_flow.append(0)
            
        else:
            positive_flow.append(0)
            negative_flow.append(0)
            
    return positive_flow, negative_flow

In [ ]:
data['Positive Flow'], data['Negative Flow'] = np.nan, np.nan

In [ ]:
data['Positive Flow'].iloc[1:],data['Negative Flow'].iloc[1:] = mf_dir(data['Typical Price'].to_numpy(), data['Money Flow'].to_numpy())

In [ ]:
data['Positive MF'] = data['Positive Flow'].rolling(rolling_period).sum()
data['Negative MF'] = data['Negative Flow'].rolling(rolling_period).sum()

In [ ]:
data['MFI'] = data['Positive MF']/ (data['Positive MF'] + data['Negative MF'])
data

In [ ]:
data[['MFI']].iloc[-52:].plot()